In [1]:
import os
import nltk
import sklearn
import gensim
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from gensim import corpora, models, similarities
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
import ndcg
import MAP
from functools import reduce
import json
import time

Using TensorFlow backend.


In [2]:
fav_books = pd.read_csv('./datasets_recsys/favorite_stories_books_sample.csv', encoding='utf-8', sep=';')

In [3]:
fav_books.head()

,user_id,story_id,franchise
0,102390,7295117,Harry Potter
1,102390,10629488,Harry Potter
2,102390,10089646,Harry Potter
3,102390,8945573,Harry Potter
4,102390,8037808,Harry Potter


In [ ]:
fav_books.shape

In [ ]:
books_info = pd.read_csv('./datasets_recsys/books_data_sample.csv', encoding='latin', sep='|')

In [ ]:
books_info['story_id'].size

In [ ]:
books_info.head()

In [ ]:
books_info = books_info[['story_title', 'author', 'franchise', 'story_id', 'story_summary']]

In [ ]:
books_info.head()

In [ ]:
print('Cantidad de historias favoritas: {}\n'.format(fav_books.shape[0]))

# Selección de usuarios para testear
Se eligió un usuario con 5 historias favoritas para test.

In [4]:
fav_books.groupby('user_id').count().sort_values(by='story_id', ascending=False).head()

,story_id,franchise
user_id,,
2835060,3295,3295
3510103,3253,3253
3562236,3224,3224
1919173,2808,2808
2938253,2543,2543


In [5]:
# Usuarios con >= 5 historias favoritas para test.
users_test_id = fav_books.groupby('user_id').count().sort_values(by='story_id', ascending=False).index.values

In [6]:
users_test_id

array([2835060, 3510103, 3562236, ..., 1941845, 1949872, 9235361])

Tokenización de las palabras: se quitan los signos de puntuación, se eliminan stopwords (palabras frecuentes sin contenido) y se podría aplicar stemm pero no lo he hecho. No creo que cambie radicalmente la recomendación.

In [ ]:
stemm = False
stemmer = PorterStemmer()

def get_tokens(text, translate_table):
    lowers = text.lower()
    no_punctuation = lowers.translate(translate_table)
    tokens = nltk.word_tokenize(no_punctuation)
    tokens = list(filter(lambda x: x not in stopwords.words('english'), tokens))
    if stemm:
        tokens = map(stemmer.stem, tokens)
    return tokens

El siguiente código crea un diccionario y puede tomar un par de minutos.

In [ ]:
translate_to = u' '
not_letters_or_digits = u'!"#%\'()*+,-./:;<=>?@[\]^_`{|}~\n'
translate_table = dict((ord(char), translate_to) for char in not_letters_or_digits)
books_info['tokenised_abstract'] = books_info.story_summary.apply(get_tokens, args=(translate_table,))

dic_file = 'lda_resources/dictionary-stemm.p' if stemm else 'lda_resources/dictionary.p'
if os.path.isfile(dic_file):
    dictionary = corpora.dictionary.Dictionary().load(dic_file)
else:
    dictionary = corpora.dictionary.Dictionary(documents=books_info.tokenised_abstract.tolist())
    dictionary.save(dic_file)

In [ ]:
stemm = False
dic_file = 'lda_resources/dictionary-stemm.p' if stemm else 'lda_resources/dictionary.p'
if os.path.isfile(dic_file):
    dictionary = corpora.dictionary.Dictionary().load(dic_file)
else:
    dictionary = corpora.dictionary.Dictionary(documents=books_info.tokenised_abstract.tolist())
    dictionary.save(dic_file)

Ejemplo de un abstract tokenizado:

In [ ]:
books_info['tokenised_abstract'][2]

Acá se cambian las palabras a vectores que hacen referencia al diccionario antes creado. Ocupa menos memoria y al final representa lo mismo.

In [ ]:
books_info['bow'] = books_info.tokenised_abstract.map(dictionary.doc2bow)
del books_info['tokenised_abstract']

In [ ]:
# corpus = books_info['bow'].tolist()

**(1)** Acá creamos la representación del corpus con TF-IDF

In [ ]:
tfidf_model_file = 'lda_resources/tfidf_model-stemm.p' if stemm else 'lda_resources/tfidf_model.p'
if os.path.isfile(tfidf_model_file):
    tfidf_model = models.tfidfmodel.TfidfModel().load(tfidf_model_file)
else:
    tfidf_model = models.tfidfmodel.TfidfModel(corpus, dictionary=dictionary)
    tfidf_model.save(tfidf_model_file)

books_info['tf_idf'] = tfidf_model[books_info.bow.tolist()]

**(2)** Acá creamos la representación del corpus con LDA usando N tópicos (Ignoren el mensaje de Warning). Intenté con 5, 10 y 50

In [ ]:
books_info.to_pickle('./datasets_recsys/LDA_model_books.pickle')
# books_info = pd.read_pickle('./datasets_recsys/LDA_model_books.pickle')

In [ ]:
books_info.head()

In [ ]:
corpus = books_info['bow'].tolist()

In [ ]:
topic_number = 20

%time lda_model = models.ldamulticore.LdaMulticore(corpus, num_topics=topic_number, workers=3)
%time books_info['lda'] = lda_model[books_info.bow.tolist()]

In [ ]:
books_info.head()

In [29]:
test = pd.read_csv('./datasets_recsys/books_info_bow.csv', encoding='latin', sep='|')

In [32]:
from ast import literal_eval
corpus = books_info['bow'].apply(lambda x: literal_eval(x))
test.bow.tolist()

['[(2676, 1), (2825, 1), (3582, 1), (4851, 1), (12622, 1)]',
 '[(670, 1), (749, 1), (856, 1), (1858, 1), (3383, 1), (3585, 1), (4328, 2), (5076, 1), (6332, 2), (24445, 1), (28692, 1), (29866, 1), (30436, 1), (40191, 1), (54595, 1)]',
 '[(61, 1), (901, 1), (2238, 1), (3804, 2), (3940, 1), (29866, 1), (43018, 3)]',
 '[(707, 1), (970, 1), (988, 1), (1000, 1), (6951, 1), (12096, 1), (12653, 1), (21998, 1), (29067, 1), (29866, 1), (45371, 1), (54595, 1)]',
 '[(87, 1), (355, 1), (506, 1), (2116, 1), (4129, 1), (9496, 1), (17107, 1), (27724, 1), (28975, 1), (29866, 1)]',
 '[(238, 1), (565, 1), (1232, 1), (1813, 1), (2593, 1), (2709, 1), (2899, 1), (3776, 1), (5701, 1), (10274, 1), (11704, 1), (21489, 1), (31514, 2), (37517, 1)]',
 '[(1461, 1), (1576, 1), (1634, 1), (1784, 1), (2003, 1), (5002, 1), (6054, 1), (8257, 1), (12027, 1), (13814, 1), (20746, 1), (29866, 1), (31335, 1), (34094, 1)]',
 '[(890, 1), (5000, 1), (5691, 1), (13859, 1), (16181, 1), (20906, 1), (29866, 1), (30092, 1), (30099,

In [34]:
corpus.tolist()

[[(2676, 1), (2825, 1), (3582, 1), (4851, 1), (12622, 1)],
 [(670, 1),
  (749, 1),
  (856, 1),
  (1858, 1),
  (3383, 1),
  (3585, 1),
  (4328, 2),
  (5076, 1),
  (6332, 2),
  (24445, 1),
  (28692, 1),
  (29866, 1),
  (30436, 1),
  (40191, 1),
  (54595, 1)],
 [(61, 1), (901, 1), (2238, 1), (3804, 2), (3940, 1), (29866, 1), (43018, 3)],
 [(707, 1),
  (970, 1),
  (988, 1),
  (1000, 1),
  (6951, 1),
  (12096, 1),
  (12653, 1),
  (21998, 1),
  (29067, 1),
  (29866, 1),
  (45371, 1),
  (54595, 1)],
 [(87, 1),
  (355, 1),
  (506, 1),
  (2116, 1),
  (4129, 1),
  (9496, 1),
  (17107, 1),
  (27724, 1),
  (28975, 1),
  (29866, 1)],
 [(238, 1),
  (565, 1),
  (1232, 1),
  (1813, 1),
  (2593, 1),
  (2709, 1),
  (2899, 1),
  (3776, 1),
  (5701, 1),
  (10274, 1),
  (11704, 1),
  (21489, 1),
  (31514, 2),
  (37517, 1)],
 [(1461, 1),
  (1576, 1),
  (1634, 1),
  (1784, 1),
  (2003, 1),
  (5002, 1),
  (6054, 1),
  (8257, 1),
  (12027, 1),
  (13814, 1),
  (20746, 1),
  (29866, 1),
  (31335, 1),
  (34094, 1

# Correr cargando los datos

Si se cargan los datos, correr desde aquí.

In [ ]:
data_path = './datasets_recsys/LDA_model_books_20_feats.csv'

books_info = pd.read_csv(data_path, encoding='latin', sep='|')

fav_books = pd.read_csv('./datasets_recsys/favorite_stories_books_sample.csv', encoding='utf-8', sep=';')
# Usuarios con >= 5 historias favoritas para test.
users_test_id = fav_books.groupby('user_id').count().sort_values(by='story_id', ascending=False).index.values

stemm = False
dic_file = 'lda_resources/dictionary-stemm.p' if stemm else 'lda_resources/dictionary.p'
if os.path.isfile(dic_file):
    dictionary = corpora.dictionary.Dictionary().load(dic_file)
else:
    dictionary = corpora.dictionary.Dictionary(documents=books_info.tokenised_abstract.tolist())
    dictionary.save(dic_file)

# corpus = books_info['bow'].tolist()

In [ ]:
books_info.head()

In [ ]:
from ast import literal_eval

books_info['lda'] = books_info['lda'].apply(lambda x: literal_eval(x))

corpus = books_info['bow'].apply(lambda x: literal_eval(x))

In [ ]:
books_info.head()

In [ ]:
books_info.shape

Esta función obtiene samples del dataset.
 - __user_id__: id del usuario objetivo que se quiere samplear.
 - __df_users__: dataframe que posee a los usuarios junto con sus historias favoritas (es necesario solo el id de la historia favorita).
 - __df_stories__: dataframe que contiene la información de las historias.
 - __p__: porcentaje de las historias favoritas del usuario objetivo para que sean utilizadas en el set de test.
 - __Nt__: número de historias tomadas de muestra para ampliar el set de test.

El objetivo de esta función es tomar las historias favoritas de un usuario al cual se le quiere hacer recomendaciones y dividirlas en dos sets:
 - __Set de training__: historias que serán usadas para buscar las historias que se recomendaráan al usuario.
 - __Set de test__: estas historias se esperan que aparezcan en la recomendación. Estas se mezclan con Nt otras historias que no han sido catalogadas como favoritas por el usuario. De esta manera, el recomendador debería recomendarle las historias del set de test.

In [ ]:
def get_samples(user_id, fav_books, books_info, p, p_favs_in_test, all_):
    """
    p: porcentaje de historias favoritas para test.
    p_favs_in_test: porcentaje de historias favoritas que estarán en test. Por ejemplo,
    si es igual a 0.1, el 10% de las historias en el set de test serán historias favoritas
    del usuario y un 90% no.
    """
    # Historias favoritas del usuario objetivo.
    user_fav_stories = fav_books[fav_books['user_id'] == user_id]
    # print(len(user_fav_stories))
    
    # Preparación samples de train y test. Aquí se separan las historias favoritas
    # del usuario en set de train y test.
    X_train, X_test = train_test_split(user_fav_stories, test_size=p, random_state=10)
    profile_train_stories = X_train['story_id']
    profile_test_stories = X_test['story_id']
    
    print('len train: ', len(profile_train_stories))
    print('len test: ', len(profile_test_stories))
    
    Nt = int(len(X_test) / p_favs_in_test) - len(X_test)

    # Sample de historias que no son favoritas del usuario objetivo.
    test_stories_nofav = books_info[~books_info['story_id'].isin(user_fav_stories['story_id'].values)]
    
    #print(test_stories_nofav)

    if not all_:
        test_samples_stories = test_stories_nofav.sample(n=Nt)

    # Se preparan las samples, esto retornará la información de las historias
    # en cada sample.
    profile_train_samples = books_info[books_info['story_id'].isin(profile_train_stories)]
    # print(len(profile_train_samples))
    #test_samples = pd.concat(
    #               [books_info[books_info['story_id'].isin(test_samples_stories)],\
    #                books_info[books_info['story_id'].isin(profile_test_stories)]])
    
    #test_samples = pd.concat([books_info[~books_info['story_id'].isin(user_fav_stories['story_id'].values)],\
    #               books_info[books_info['story_id'].isin(profile_test_stories)]])
                                      
    test_samples = pd.concat([test_samples_stories, \
                              books_info[books_info['story_id'].isin(profile_test_stories)]])

    print('len test sample: ', len(test_samples))
    # print(len(df_stories[df_stories['story_id'].isin(test_samples_stories)]))
    # print(len(df_stories[df_stories['story_id'].isin(profile_test_stories)]))
    return profile_train_samples, test_samples

In [ ]:
# Se seleccionó el usuario que más historias favoritas tiene: 5.
user_id = users_test_id[0]
print('Usuario: {}'.format(user_id))
print(len(books_info))
%time train_samples, test_samples = get_samples(user_id, fav_books, books_info, 0.3, 0.05, False)


In [ ]:
print(len(test_samples))
print(len(train_samples))

# Test del split en train y test

## Historias favoritas del usuario objetivo.

In [ ]:
fav_books[fav_books['user_id'] == user_id]

## Train de las historias favoritas.

In [ ]:
train_samples

## Test de las historias favoritas
Se tuvo que buscar entre las (Nt + porcentaje-de-historias-favoritas-para-test) para ver si correctamente se añadieron al set de test.

In [ ]:
test_samples[test_samples['story_id'].isin(fav_books[fav_books['user_id'] == user_id]['story_id'])]

#### El siguiente es una función necesaria para procesar LDA

In [ ]:
def to_sparse(matrix, N):
    return csr_matrix([gensim.matutils.sparse2full(row, length=N) for row in matrix]) 

# Configuraciones del modelo a utilizar.

In [ ]:
model = 'lda' # 'lda' 'tf_idf'
nearest_neighbors = 10
metric = 'euclidean' # 'cosine' 'euclidean'
M = len(corpus)
N = len(dictionary)

print(M,N)

#X = to_sparse(test_samples[model].tolist())
#document_index = NearestNeighbors(n_neighbors=(nearest_neighbors+1),\
                              ##    algorithm='ball_tree', metric=metric, n_jobs=-1).fit(X)

In [ ]:
def get_recommendations(model, q_recommendations, metric, N_dict, X_train, X_test):
    X = to_sparse(X_test[model].tolist())
    document_index = NearestNeighbors(n_neighbors=(nearest_neighbors+1),\
                                      algorithm='brute', metric=metric).fit(X)
    dense_train_avg = np.array([gensim.matutils.sparse2full(row, length=N_dict) for row \
                                in X_train[model].tolist()]).mean(axis=0)
    dists, neighbors = document_index.kneighbors(dense_train_avg.reshape(1, -1))
    stories = []
    for i in neighbors[0]:
        stories.append(X_test.iloc[i]['story_id'])
    return stories

In [ ]:
def recommend_stories_users(users, fav_books, books_info, p, Nt, **kwargs):
    recommendation_dict = {}
    for user in users:
        t1 = time.time()
        train_samples, test_samples = get_samples(user, fav_books, books_info, p, Nt, False)
        recommendation = get_recommendations(kwargs['model'], kwargs['nearest_neighbors'],\
                                           kwargs['metric'], kwargs['N'], train_samples,\
                                           test_samples)
        recommendation_dict[user] = recommendation
        # print('finish')
        t2 = time.time()
        print('Finished user {} in {}'.format(user, t2 - t1))
    with open('./{}_{}_{}.json'.format(kwargs['model'], kwargs['nearest_neighbors'],\
                                      kwargs['metric']), 'w') as fp:
        json.dump(recommendation_dict, fp)
    return recommendation_dict
    
        

In [ ]:
args = {
    'model': 'lda',
    'nearest_neighbors': 10,
    'metric': 'euclidean',
    'N': len(dictionary)
}
users = fav_books['user_id'].unique()
test_users = np.random.choice(users, 1000)
%time dic = recommend_stories_users(test_users, fav_books, books_info, 0.3, 0.05, **args)

In [ ]:
dic

In [ ]:
recommended_stories = get_recommendations(model, 10, metric, N, train_samples, test_samples)

In [ ]:
for story in recommended_stories:
    info = test_samples[test_samples['story_id'] == story].iloc[0]
    print('{} ({})'.format(info['story_title'], info['franchise_x']))
    print(info.story_summary)
    print('\n\n')

Por cada historia favorita en el set de train se buscan las "nearest_neighbors" historias más similares, las cuales son printeadas abajo. Lamentablemente no aparece ninguna de las dos historias que se añadieron al set de test. Esto puede deberse porque Harry Potter está muy saturado.

Ideas para mejorar esto es sacar el promedio de los vectores de las historias favoritas del usuario que fueron utilizadas para test. De esta forma se buscarán solo "nearest_neighbors" historias favoritas y no "nearest_neighbors" historias para cada historia favorita del usuario.

### Aqui obtenemos las metricas de ndcg10, MAP, y precision@10 para el usuario objetivo id 4446515 respecto a relevancia de franquicias: 

In [ ]:

results = []

cont = 0 

for n, (ix, paper) in enumerate(train_samples.iterrows()):
    dists, neighbors = document_index.kneighbors(gensim.matutils.sparse2full(paper[model],length=N).reshape(1, -1))
    

for x in neighbors[0]:
    results.append([fan_df_stories.iloc[x]['franchise_x'], dists.tolist()[0][cont]])
    cont+=1

results.sort(key=lambda x: x[1], reverse=True)

franchises = [x[0] for x in results]



In [40]:
import pickle
dic = pickle.load(open('./lda_15_cosine.pickle', 'rb'))

In [41]:
results = []
for key, value in dic.items():
    relevantes = []
    for x in value:
        books_relev = list(fav_books[fav_books['user_id'] == int(key)]['story_id'])
        # print('Len relevantes: {}'.format(len(books_relev)))
        if x in books_relev:
            relevantes.append(1)
        else:
            relevantes.append(0)

    ndcg10 = ndcg.ndcg(relevantes,10)
    precision10 = MAP.precision_at_k(relevantes,10)
    lista_MAP = [MAP.precision_at_k(relevantes,x) for x in range(1, len(relevantes))]
    mean_average_precision = reduce(lambda x, y: x + y, lista_MAP) / len(lista_MAP)

    #print('resultados para el usuario objetivo de id: {}'. format(key))
    #print('ndcg@10: {}'.format(ndcg10))
    #print('precision@10: {}'.format(precision10))
    #print('MAP: {}'.format(mean_average_precision))
    results.append([key, ndcg10, precision10, mean_average_precision])
df_results = pd.DataFrame(results, columns=['user_id', 'ndcg10', 'precision10', 'MAP'])
    



In [42]:
df_results.describe()

,user_id,ndcg10,precision10,MAP
count,1.867000e+03,1867.000000,1867.000000,1867.000000
mean,3.316013e+06,0.376597,0.132405,0.143492
std,2.281332e+06,0.308913,0.135795,0.174523
min,1.970500e+04,0.000000,0.000000,0.000000
25%,1.572261e+06,0.000000,0.000000,0.000000
50%,2.632396e+06,0.386853,0.100000,0.084563
75%,4.768792e+06,0.572503,0.200000,0.214008
max,9.227721e+06,1.000000,0.900000,0.990000
